# BM25 Algorithm
BM25 is a probabilistic algorithm for information retrieval that is described in [1]. It is summarized here.

## 1. Msmarco Dataset
We'll use the [msmarco](https://microsoft.github.io/msmarco/TREC-Deep-Learning-2019) dataset for examples in this notebook.

In [2]:
!head ../data/msmarco/msmarco-docs.tsv -n 1

D1555982	https://answers.yahoo.com/question/index?qid=20071007114826AAwCFvR	The hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.?	Science & Mathematics Physics The hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.? It is a good approximation to assume that the emissivity e is equal to 1 for these surfaces. Find the radius of the star Rigel, the bright blue star in the constellation Orion that radiates energy at a rate of 2.7 x 10^32 W and has a surface temperature of 11,000 K. Assume that the star is spherical. Use σ =... show more Follow 3 answers Answers Relevance Rating Newest Oldest Best Answer: Stefan-Boltzmann law states that the energy flux by radiation is proportional to the forth power of the temperature: q = ε · σ · T^4 The total energy flux at a spherical surface of Radius R is Q = q·π·R² = ε·σ·T^4·π·R² Hence the radius is R = √ ( Q / (ε·σ·T^4·π) ) = √ ( 2.7x10+32 W / (1 · 5.67x10-8W/m²K^4 · (1100K)^4 · π) 

Each line of the `msmarco-docs.tsv` contains one document, with four fields per document: document ID, URL, title, and body.

The `msmarco-docs.tsv` file contains 3.2 million documents in one 22GB file, which is cumbersome to work with.

In [1]:
!wc ../data/msmarco/docs/msmarco-docs.tsv -l

3213835 ../data/msmarco/msmarco-docs.tsv


The Linux *split* command will split the TSV file into 64 documents, each containing 50,000 documents and several hundred MB in size.

In [5]:
#! split -d -l 50000 ../data/msmarco/msmarco-docs.tsv
! ls -l -h | head

total 22G
-rw-rw-r-- 1 ubuntu ubuntu  24K May 25 00:18 bm25.ipynb
drwxrwxr-x 2 ubuntu ubuntu 4.0K May 24 04:12 wordpiece
-rw-rw-r-- 1 ubuntu ubuntu 339M May 24 03:46 x00
-rw-rw-r-- 1 ubuntu ubuntu 347M May 24 03:46 x01
-rw-rw-r-- 1 ubuntu ubuntu 337M May 24 03:46 x02
-rw-rw-r-- 1 ubuntu ubuntu 342M May 24 03:46 x03
-rw-rw-r-- 1 ubuntu ubuntu 335M May 24 03:46 x04
-rw-rw-r-- 1 ubuntu ubuntu 340M May 24 03:46 x05
-rw-rw-r-- 1 ubuntu ubuntu 345M May 24 03:46 x06


## 2. Simple IDF-ish Algorithm
### A. Math
Eq (1)
$$P(\textrm{rel}|d, q) \propto_q \frac{P(\textrm{rel}|d, q)}{P(\overline{\textrm{rel}}|d, q)}$$

$P(\textrm{rel}|d, q)$ is the probability that document $d$ is relevant for query $q$. Eq(1) shows that the probability is proportional to the odds ratio, which is the probability that the document is relevant over the probability that the document is not relevant.

Next, a lot of math and approximations happen and we end up with Eq (2). See section 2.4 of [1] for details.

Eq(2)
$$P(\textrm{rel}|d, q) \propto_q \sum_{\textbf{q}, tf_i > 0} w_i$$

$w_i$ is the weight for term $t_i$, which represents the importance of term $i$ in document $d$. We calculate the score of a document by summing the weights of all terms that appear in both the query $q$ and the document. We can then return the top $m$ documents in descending order of $P(\textrm{rel}|d, q)$.

A simple way of determining $w_i$ is to use Eq(3), which corresponds to equation 3.3 in [1]. This approach is very similar to a classical *idf* approach.

Eq(3)
$$w_i^{\textrm{IDF}} = \log\frac{N - n_i + 0.5}{n_i + 0.5}$$

N is the number of documents in the corpus and $n_i$ is the number of documents in the corpus that contain term $t_i$. For terms that appear in most documents, $n_i$ will be close to $N$ and $w_i^{\textrm{IDF}}$ will be close to zero. The values of $0.5$ in the numerator and denominator smooth the weights in the event of rare query terms.

There are about 3.2 million documents in the corpus. Working with a single 22GB file is problematic. Splitting it into files with 50,000 docs each.

### B. Information Retrieval Toy Example

In [9]:
import math

import nltk
from nltk.corpus import stopwords
import pandas as pd
import pickle
from tokenizers import BertWordPieceTokenizer

We will index and create a retrieval system for a corpus of 1000 documents selected from an arbitrary text file.

In [149]:
doc_list = pd.read_csv("x10", sep="\t", header=None, names=["doc_id", "url", "title", "doc"], nrows=1000)
doc_list

,doc_id,url,title,doc
0,D2273143,https://www.allaboutcircuits.com/textbook/refe...,Solving Simultaneous Equations,Solving Simultaneous Equations Chapter 4 - Alg...
1,D1029983,http://xynyth.com/resource/icemelter-concrete/...,.,"Icemelters and Concrete- The basics, what ever..."
2,D312314,http://medical-dictionary.thefreedictionary.co...,nurse's aide,"nurse's aide Also found in: Dictionary, Thesau..."
3,D582292,https://www.business-case-analysis.com/cost-al...,Cost Allocation and Cost Apportionment Definit...,Home > Encyclopedia > C > Cost Allocation Cost...
4,D1021364,http://mexicanspanish.com/equis/,Equis,"Wednesday, January 27, 2016 by Mark Robert Ale..."
...,...,...,...,...
995,D2634529,https://www.quora.com/What-is-the-difference-b...,What is the difference between FedEx Standard ...,"Emily Baker, Previous Fed Ex Office Manager An..."
996,D728098,https://answers.yahoo.com/question/index?qid=2...,How many animals are killed per day?,Pets Other - Pets How many animals are killed ...
997,D3381877,http://www.calculator.pro/Body_Mass_Index.html,Body mass index: BMI calculator for women and men,"Enter all data for best result Imperial (lb, f..."
998,D3224131,https://www.technorms.com/42603/cancel-netflix...,How to Cancel Your Netflix Service Subscription,Home Web How to Cancel Your Netflix Service Su...


We would like to remove stopwords form our index and queries.

In [150]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

This class creates a simple inverse document frequency index and provides the ability to search the index.

In [165]:
class Index():
    """Creates a term index based on inverse document frequency (IDF)
    
    The fomat of self.index is [n, weight, set(doc_ids)]
    n is the number of documents that contain the search term.
    """
    def __init__(self, doc_list):
        self.doc_list = doc_list
        self.index = {} # term: [n, weight, set(doc_ids)]
        self.tokenizer = BertWordPieceTokenizer(
            "wordpiece/bert-base-uncased-vocab.txt", lowercase=True)
        self.N = 0
        self.stopwords = set(stopwords.words('english'))
        self.stopwords.add("[CLS]")
        self.stopwords.add("[SEP]")
        
        self.add_docs()
        
    def add_docs(self):
        for row in self.doc_list.itertuples(False):
            self.add_doc(row[0], row[3])
        self.update_weights()
        
    def add_doc(self, doc_id, doc):
        """Adds a single document to the index.
        
        Since the index stores the number of matching documents,
        addition documents can be added to the index at any
        time.
        """
        if not isinstance(doc, str):
            return
        encoding = self.tokenizer.encode(doc)
        tokens = set(encoding.tokens)
        for tkn in tokens:
            self._add_term(tkn, doc_id)
        self.N += 1
            
    def _add_term(self, term, doc_id):
        """Adds a single term to the index.
        """
        if term in self.stopwords:
            return
        if term in self.index:
            if doc_id not in self.index[term][2]:
                self.index[term][0] += 1
                self.index[term][1] = None
                self.index[term][2].add(doc_id)
        else:
            self.index[term] = [1, None, set([doc_id])]
            
    def _calc_weight(self, n):
        """Calculates an IDF weight for a term using Eq(3)."""
        return math.log((self.N - n + 0.5)/(n + 0.5))
    
    def update_weights(self):
        """Updates all weights in the index."""
        for entry in self.index.values():
            entry[1] = self._calc_weight(entry[0])
        self.weight_df = pd.DataFrame(
            [{"term": key, "count": val[0], "weight": val[1]}
             for key, val in self.index.items()])
        
    def search(self, query):
        """Accepts a search query and returns matching documents."""
        # Break query into WordPiece tokens
        query_tokens = [tkn for tkn in self.tokenizer.encode(query).tokens
                       if tkn not in self.stopwords]
        token_doc_pairs = []

        # Emit a doc ID, weight, and corresponding token for each
        #   document with a term that occurs in the query.
        #   (Map phase of mapReduce)
        for tkn in query_tokens:
            for doc_id in self.index[tkn][2]:
                token_doc_pairs.append([doc_id, self.index[tkn][1], [tkn]])
                
        token_doc_pairs.sort(key=lambda x: x[0])
        
        # Combine weights for same documents
        #   (Combine phase)
        results = []
        current_pair = token_doc_pairs[0]
        for next_pair in token_doc_pairs[1:]:
            if next_pair[0] == current_pair[0]:
                current_pair = [current_pair[0], current_pair[1] + next_pair[1],
                               current_pair[2] + next_pair[2]]
            else:
                results.append(current_pair)
                current_pair = next_pair
        results.append(current_pair)
        
        # Put results in a Dataframe
        results.sort(key=lambda x: x[1], reverse=True)
        for result in results:
            result.append(
                self.doc_list.loc[self.doc_list.doc_id == result[0],
                                  "title"].values[0])
        results_df = pd.DataFrame(results, columns=["Doc ID",
                                                    "Summed Weights",
                                                    "Matching Terms",
                                                    "Title"])
        return results_df

Creating the index from the document list.

In [166]:
idf = Index(doc_list)

A simple query

In [178]:
idf.search("apple watch tips").head(10)

,Doc ID,Summed Weights,Matching Terms,Title
0,D1530634,8.501399,"[apple, watch, tips]",Symptoms of Early Menopause
1,D3010006,8.501399,"[apple, watch, tips]",41 Apple Watch tips and tricks
2,D1520949,6.013773,"[apple, tips]","Explore Coal Miners, West Virginia, and more!"
3,D1557363,6.013773,"[apple, tips]",Cecropia Moth - Hyalophora cecropia
4,D2060971,6.013773,"[apple, tips]",Mango
5,D241774,6.013773,"[apple, tips]",The 40 Best Low-Calorie Foods
6,D2534813,6.013773,"[apple, tips]","""2017 Winner of Best Education Podcast in the ..."
7,D79212,6.013773,"[apple, tips]",Do Try This At Home: Hacking Ribs â In The Pre...
8,D1617231,5.846653,"[apple, watch]",'It's another one of his outlandish moves': Te...
9,D1794341,5.846653,"[apple, watch]",Aslaug


* The number of times a term appears in a document has no impact on the search results.
* A matching term contributes the same weight, no matter what document it appears in.

## Citations
[1] Stephen E. Robertson and Hugo Zaragoza. 2009. The Probabilistic Relevance
Framework: BM25 and Beyond. [Foundations and Trends in Information Retrieval
(2009).](https://drive.google.com/file/d/1ni_fbufB4irOJTIRzunNQFD23MotEpT7/view?usp=sharing)